# 模型类

到目前为止，我们已经先后创建了张量、层、损失函数、优化器、和数据集等五个核心基础类。

现在，我们将创建最后一个核心基础类：**模型**（Model）。我们将用模型把层、损失函数、和优化器，以及模型训练和模型测试的逻辑，全部封装到一起。

```{figure} images/model.png
:align: center
:width: 360px
**图例：框架分层结构**

* **顶层**：模型（流程控制）；
* **中层**：层、损失函数、优化器（逻辑执行）；
* **底层**：张量（自动微分）。

```

In [1]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return self.data.shape[-1]

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础数据集

In [3]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size

        self.test_labels = self.test_features = None
        self.train_labels = self.train_features = None

        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    @property
    def shape(self):
        return Tensor(self.features).size, Tensor(self.labels).size

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

### 基础层

In [4]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    @property
    def parameters(self):
        return []

    def __repr__(self):
        return ''

### 基础损失函数

In [5]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

In [6]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### 基础模型

**基础模型**（Base Model）是一个抽象类，封装了两个虚拟接口：

* **训练函数**（train）：训练模型；
* **测试函数**（test）：测试模型。

创建一个模型，我们需要提供**层**、**损失函数**和**优化器**作为参数。

In [7]:
class Model(ABC):

    def __init__(self, layer, loss_fn, optimizer):
        self.layer = layer
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## 数据

### 数据集

In [8]:
class Dataset(Dataset):

    def load(self):
        self.train_features = [[22.5, 72.0],
                               [31.4, 45.0],
                               [19.8, 85.0],
                               [27.6, 63.0]]
        self.train_labels = [[95],
                             [210],
                             [70],
                             [155]]

        self.test_features = [[28.1, 58.0]]
        self.test_labels = [[165]]

## 模型

### 线性层

In [9]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data
            self.bias.grad += np.sum(p.grad, axis=0)
            x.grad += p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    @property
    def parameters(self):
        return [self.weight, self.bias]

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 损失函数（均方误差）

In [10]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data) / len(y.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 优化器（随机梯度下降）

In [11]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### 神经网络模型

我们的第一个模型类是**神经网络模型**（NNModel）。它把我们之前的模型训练和模型测试逻辑封装在一起，这样我们只需要简单调用就可以进行模型的训练和测试。

同时，这里我们引入**轮次**（Epoch）的概念。

In [12]:
class NNModel(Model):

    def train(self, dataset, epochs):
        # 数据集训练模式
        dataset.train()

        # 轮次
        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                loss = self.loss_fn(predictions, labels)
                self.optimizer.reset()
                loss.backward()
                self.optimizer.step()

    def test(self, dataset):
        # 数据集测试模式
        dataset.eval()

        features, labels = dataset.items()
        predictions = self.layer(features)
        loss = self.loss_fn(predictions, labels)
        return predictions, loss

## 设置

### 学习率

In [13]:
LEARNING_RATE = 0.00001

### 批大小

In [14]:
BATCH_SIZE = 2

### 轮次

In [15]:
EPOCHS = 1000

## 训练

### 建模

建模的过程没有太大的变化，依旧是分别创建数据集、层、损失函数和优化器。最后使用他们作为参数创建模型。

In [16]:
dataset = Dataset(BATCH_SIZE)

layer = Linear(dataset.shape[0], dataset.shape[1])
loss_fn = MSELoss()
optimizer = SGDOptimizer(layer.parameters, lr=LEARNING_RATE)

model = NNModel(layer, loss_fn, optimizer)
print(model.layer)

Linear[weight(1, 2); bias(1,)]


### 迭代

通过创建一个模型，把数据集、层、损失函数、和优化器结合起来以后。模型训练的代码简单明了。

In [17]:
model.train(dataset, EPOCHS)

## 验证

### 测试

模型测试的代码也同样简洁。

In [18]:
predictions, loss = model.test(dataset)
print(f'prediction: {predictions}')
print(f'loss: {loss}')

prediction: Tensor([[163.52327795]])
loss: Tensor(2.1807080003282495)


到此为止，我们完成了我们的**神经网络训练框架**核心架构的构建。

## 课后练习

尝试在模型类中添加**早停**和**重排**功能。